In [77]:
from datetime import date
from bs4 import BeautifulSoup as soup
import requests
import tqdm.notebook as tq
import random
from datetime import datetime
import calendar

In [79]:
def scrape_cna():
    """
    Scrape Channel News Asia Topic Terrorism
    """   
    # Base URL
    cna_url="https://www.channelnewsasia.com/topic/terrorism"
    cna_base_url = 'https://www.channelnewsasia.com'

    # Get webpage
    html = requests.get(cna_url)

    # Initialise bs object
    bsobj = soup(html.content,'lxml')

    # Find all headers
    headers = bsobj.findAll("h6")

    # initialize the progress bar
    # select a random color
    colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white', 'steelblue']
    random_color = random.choice(colors)
    loop = tq.tqdm(enumerate(headers), total=len(headers), 
                        leave=True, colour=random_color, unit='article')

    articles = []

    for _, header in loop:
        item = {
            'headline': header.text
        }

        # Enter news article
        news_link = cna_base_url + header.a['href']
        news_html = requests.get(news_link)

        # Traverse news article for main body
        article = soup(news_html.content,'lxml')
        content = article.find('div', {'class': 'content'})
        if content:
            content_wrapper = content.find_all('div', {'class': 'content-wrapper'})

            article_text = []
            for content in content_wrapper:
                # Extract text
                main_text = content.find('div', {'class': 'text'})
                if main_text:
                    article_text.append(main_text.get_text())
        
            item['text'] = article_text
            articles.append(item)
        
        # Get timestamp
        timestamp_element = article.find('div', {'class': 'article-publish'})
        if timestamp_element:
            timestamp = timestamp_element.get_text(separator='|', strip=True).split('|')[0]
            timestamp_obj = datetime.strptime(timestamp, "%d %b %Y %I:%M%p")
            item['timestamp_published'] = calendar.timegm(timestamp_obj.timetuple())
        
        loop.set_postfix(Processing=header.text)

    return articles

def format_articles(articles):
    """
    Format scraped data
    """

    formatted_articles = []
    
    # Format article
    for article in articles:
        if 'text' not in article:
            continue
        
        article = {
            'title': article['headline'],
            'body': "\n".join(article['text']),
            'timestamp_published': article['timestamp_published'] if 'timestamp_published' in article else 'null'
        }

        formatted_articles.append(article)

    return formatted_articles

def scrape_apnews():
    """
    Scrape AP News Topic Terrorism
    """

    # Base URL
    cna_url="https://apnews.com/hub/terrorism"

    # Get webpage
    html = requests.get(cna_url)

    # Initialise bs object
    bsobj = soup(html.content,'lxml')

    # Find all article item in first section
    items = bsobj.findAll("h3", {'class':'PagePromo-title'})

    # initialize the progress bar
    # select a random color
    colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white', 'steelblue']
    random_color = random.choice(colors)
    loop = tq.tqdm(enumerate(items), total=len(items), 
                        leave=True, colour=random_color, unit='article')
    
    articles = []

    for _, header in loop:
        item = {
            'headline': header.text
        }

        # Enter news article
        news_link = header.a['href']
        news_html = requests.get(news_link)

        # Traverse news article for main body
        article = soup(news_html.content,'lxml')
        content = article.find('div', {'class': 'RichTextStoryBody'})
        if content:
            article_texts = content.find_all('p')
            article_text = []
            for text in article_texts:
                article_text.append(text.text)
            item['text'] = article_text

        timestamp_element = article.find('bsp-timestamp')
        if timestamp_element:
            item['timestamp_published'] = timestamp_element['data-timestamp']

        articles.append(item)
        
        # Update progress description
        loop.set_postfix(Processing=header.text)
    
    return articles

In [55]:
articles = scrape_apnews()
formatted_articles_apnews = format_articles(articles)

  0%|          | 0/39 [00:00<?, ?article/s]

In [58]:
print(formatted_articles_apnews[2].keys())

dict_keys(['title', 'body', 'timestamp_published'])


In [80]:
articles = scrape_cna()
formatted_articles_cna = format_articles(articles)

  0%|          | 0/15 [00:00<?, ?article/s]

In [81]:
print(formatted_articles_cna[12])

{'title': "\n\n      CNA Explains: What happens if there's a bomb threat on your flight?\n  \n", 'body': '\nSINGAPORE: Earlier this month, a bomb threat on Scoot flight TR16\xa0forced the Perth-bound plane to turn back to Singapore.\nAn Australian passenger was arrested and charged with making false threats of terrorist acts. He allegedly repeatedly told cabin crew that he had a bomb.\nLast year, Singapore Airlines flight SQ33 from San Francisco also had a similar scare, when a bomb threat was made more than 12 hours after take-off.\nThe plane eventually landed safely at Changi Airport and the passenger was given a stern warning and jail for slapping a cabin crew member.\nThe Republic of Singapore Air Force\'s (RSAF) Air Defence Task Force deals with more than 350 suspicious air threats each year on average, according to\xa0a 2022 article from the Singapore Armed Forces PIONEER magazine.\nWhat can passengers expect after a bomb threat is made on a plane, and how do security efforts tak